In [1]:
import pandas as pd
from sklearn.metrics import r2_score
from datetime import datetime

In [3]:
df=pd.read_excel('gpb_adm_data.xlsx', sheet_name='Incomes').set_index('TID')
df=df.rename(columns={c:str(c).split(' ')[0] for c in df.columns})

In [5]:
col_set_nov=[f'2022-11-{i:0>2}' for i  in range(15, 31)]
col_set_sept=[f'2022-09-{i:0>2}' for i  in range(1, 16)]
target_df=(df[col_set_nov].apply(sum, axis=1)/(df[col_set_nov]>0).astype(int).apply(sum, axis=1)
          ).rename('target_nov').to_frame()
target_df=target_df.join((df[col_set_sept].apply(sum, axis=1)/(df[col_set_sept]>0).astype(int).apply(sum, axis=1)
          ).rename('target_sept'))
target_df=target_df[target_df.target_sept.notnull()&target_df.target_nov.notnull()]
target_df.to_csv('target_nov_sept.csv')

In [7]:
r2_score(target_df.target_sept, target_df.target_nov)

0.8391220853117868

In [9]:
sept_days={i: [] for i in range(7)}
for i  in range(1, 16):
    sept_days[datetime(2022, 9, i).weekday()].append(f'2022-09-{i:0>2}')
nov_days={i: [] for i in range(7)}
for i  in range(15,31):
    nov_days[datetime(2022, 11, i).weekday()].append(f'2022-11-{i:0>2}')
week_days=['Пн', 'Вт', 'Ср', 'Чт', 'Пт', 'Сб', 'Вс'] 
for i in range(7):
    x=df[sept_days[i]].apply(sum, axis=1)/(df[sept_days[i]]>0).astype(int).apply(sum, axis=1)
    y=df[nov_days[i]].apply(sum, axis=1)/(df[nov_days[i]]>0).astype(int).apply(sum, axis=1)
    print(week_days[i], r2_score(x[x.notnull()&y.notnull()], y[x.notnull()&y.notnull()]))

Пн 0.7181345934172241
Вт 0.703390845963458
Ср 0.8009815204759457
Чт 0.4962865524503747
Пт 0.5017266892174792
Сб 0.7448523623491778
Вс 0.6932683643568567
